Esta es una clase abstracta clase SensorNetworksProblem que tu definas tiene que estar basada en esta

In [2]:
class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

Esta clase Node crea un arbol de vecindarios a partir de un estado inicial y te permite trackear los vencindarios padre. Seguramente no requiere ninguna modificación de tu parte y solo es utilizada en la función simulated_annealing

In [ ]:
## Esta clase probablemente no tienes que modificarla

class Node:

    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state.  Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node.  Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def __lt__(self, node):
        return self.state < node.state

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action,
                    problem.path_cost(self.path_cost, self.state,
                                      action, next_state))
        return next_node
    
    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # We want for a queue of nodes in breadth_first_graph_search or
    # astar_search to have no duplicated states, so we treat nodes
    # with the same state as equal. [Problem: this may not be what you
    # want in other contexts.]

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        return hash(self.state)

Esta función define el algoritmo de simulated annealing y emplea un problema que definas tu y la case Node

In [ ]:
## No debería haber necesidad de que alteraras este código

def simulated_annealing(problem, schedule=exp_schedule()):
    """[Figure 4.5] CAUTION: This differs from the pseudocode as it
    returns a state instead of a Node."""
    current = Node(problem.initial)
    for t in range(sys.maxsize):
        T = schedule(t)
        if T == 0:
            return current.state
        neighbors = current.expand(problem)
        if not neighbors:
            return current.state
        next_choice = random.choice(neighbors)
        delta_e = problem.value(next_choice.state) - problem.value(current.state)
        if delta_e > 0 or probability(math.exp(delta_e / T)):
            current = next_choice

Esta función imprime el grid de acuerdo al libro (del lado izquierdo)

Para sacar el del lado derecho puedes basarte en esta función

In [ ]:
    def printgrid(self, rows):
        i = 1

        for row in rows:
            if i == 1:
                print(" ", " ".join([str(i + 1) for i in range(len(rows))]))
         
            print(i, " ".join([ '-' if cell == 1 else 'X' for cell in row  ]))
            i += 1

Esta es una implementación de un problema. Nota que esta "sobrecargando" los métodos de la clase abstracta Problem.

Tu tendrías que definir tu propia clase SensorNetworksProblem de un modo muy parecido a esta

In [ ]:
# Implementation Example
# Linea 724 search.py

class PeakFindingProblem(Problem):
    """Problem of finding the highest peak in a limited grid"""

    def __init__(self, initial, grid, defined_actions=directions4):
        """The grid is a 2 dimensional array/list whose state is specified by tuple of indices"""
        Problem.__init__(self, initial)
        self.grid = grid
        self.defined_actions = defined_actions
        self.n = len(grid)
        assert self.n > 0
        self.m = len(grid[0])
        assert self.m > 0

    def actions(self, state):
        """Returns the list of actions which are allowed to be taken from the given state"""
        allowed_actions = []
        for action in self.defined_actions:
            next_state = vector_add(state, self.defined_actions[action])
            if next_state[0] >= 0 and next_state[1] >= 0 and next_state[0] <= self.n - 1 and next_state[1] <= self.m - 1:
                allowed_actions.append(action)

        return allowed_actions

    def result(self, state, action):
        """Moves in the direction specified by action"""
        return vector_add(state, self.defined_actions[action])

    def value(self, state):
        """Value of a state is the value it is the index to"""
        x, y = state
        assert 0 <= x < self.n
        assert 0 <= y < self.m
        return self.grid[x][y]

Para llegar a tu clase de SensorNetworksProblem tienes que poner mucha atención en los actions. Porque a diferencia del PeakFindingProblem tu estas moviendo N sensores de distinta cobertura sobre el grid. Recuerda que el método actions te devuelve una lista las acciones válidas

In [ ]:
class SensorNetworksProblem(Problem):
    def __init__(self, initial, sensors_range, grid, defined_actions):
        super().__init__(initial)
        # Defines grid
    
    def actions(self, state):
        """ Defines possible actions from the current state"""
        # Mover sensor izquierda, arriba abajo. Validar que es posible porque no esta fuera del grid
        # o que los sensores esten "encimados"
        # return allowed_actions
        # una lista de las combinaciones de las acciones aplicadas para cada sensor
        
        
    def result(self, state, action):
        """ Toma un estado y aplica la acción determinada """
       # return applyaction(state,action)
    def value(self, state):
        """ Evalua un estado """

        # 
        
        # Aquí tienes que hacer print del formato especificado en el ejercicio para la solución (state)
        # 123456 123456 1--XX-X 111#X2# 2-XX–-- 21##–22 3XX---- ==> 3##1--- 4------ 4----33 5XXX--- 5XX#-33 6----XX 6----XX
        
        # return valor objetivo es la cuenta de el número de "unos" del grid estan cubriendo los sensores
        # Un modo de hacerlo es iterar sobre cada una de las "celdas" del grid y preguntar si ahí hay un sensor
        

Para ser utilizado necesitas un estadio inicial de los sensores, el grid sobre el que vas a trabajar y las acciones, que básicamente es mover los sensores en los puntos cardinales

In [ ]:
sensors_range = [3,2,2,1]
initial = [(1,1), (1,5), (4,5), (5,3)] # Dónde empiezan nuestros 4 sensores
grid = [[1,1,0,0,1,0], [1,0,0,1,1,1], [0,0,1,1,1,1], [1,1,1,1,1,1], [0,0,0,1,1,1], [1,1,1,1,0,0]] #definido en el ejercicio
defined_actions = {'E': (1, 0), 'N': (0, 1), 'S': (0, -1), 'W': (-1, 0)}





myproblem = SensorNetworksProblem(initial, sensors_range, grid, defined_actions)

solutions = {problem.value(simulated_annealing(problem)) for i in range(100)}
max(solutions)